# **Triangular Arbitrage Finder**

**What is arbitrage?**

Arbitrage is a discrepancy between the prices of two (or more) assets that allows making risk-free profit. For instance, if I was prepared to pay USD1 for an apple, and you know you can get one from the store for 50 cents, the rational thing for you to do is to purchase one from the store and sell it to me, and net 50 cents in the process. A similar logic applies on the stock market. For example, the stock A trades for USD100 on the NYSE while, at the same time, it trades for USD101 at the Shanghai Stock Exchange. You can buy the stock from the NYSE and immediately sells it on the Shanghai market, earning a profit of USD1

Notice, however, that buying the stock will exert upward pressure on the stock’s price on NYSE, and similarly, selling will impose downward price pressure on Shanghai market. As a result of this dynamic, the spread in the stock’s prices on the two marketplaces will narrow down until arbitrage is unprofitable. Due to the vast number of investors scanning these exploitable price discrepancies on a sub-second scale, they are rare and short-lived in a liquid market such as the stock market, or the cryptocurrency market for that matter. 

**What is triangular arbitrage?**

Triangular arbitrage is the result of a discrepancy between three assets - often currencies - that occurs when the currencies exchange rates do not exactly match up. This enables the trader to trade the first currency to the second, the second to third, and the third back to the first, so that she nets a small profit in the process. The following example may help understanding the idea:

Let us assume you have $1 million, and you observe the following exchange rates on the market: USD/EUR = 0.8698; GBP/EUR = 1.3021; and GBP/USD = 1.5028. Can you make money through triangular arbitrage? Yes, you can, through the following operations.

* Sell dollars for euros: USD1,000,000 x 0.8698 = EUR869,800
* Sell euros for pounds: EUR869,800 / 1.3021 = GBP667,997.85
* Sell pounds for dollars: GBP667,997.85 x 1.5028 = USD1,003,867.17

You started with USD1 million, and ended up with USD1,003,867.17, implying the net profit was USD3,867.17, or roughly 0.39%. Notice, that it does not matter which of the three currencies is the "starting currency" held in beginning, as long as the trades are the same and come in same order in the loop.

Now, you may think that a USD4k profit a abysmal considering you had USD1 million capital, but notice that you made it instantaneously, and thus, you can potentially make hundreds of such trades in a day. Indeed, making frequent trades with large capital in hopes of small profits is the essence of arbitrage trading.

Now, it should go without saying that in practice, arbitrage of any kind is not the kind of money printing machine that the example above would you make you think. There many, many factors that make arbitrage trading *significantly* more challenging in practice than on paper. Among the main obstacles are fees and bid-ask spreads (though they are accounted for in my implementation), the sheer number of professional, highly optimized, lightning-fast algorithms competing for the same marginal profit, and rapid price fluctuations.

**This project**

This notebook contains a triangular arbitrage finder for the cryptocurrency market. It scans continuously the prices of all cryptocurrency pairs traded on Binance through a websocket, and reports back when it has identified a profitable triangular arbitrage opportunity with instructions about which currencies to trade and at which price. In other words, it automates the process of finding opportunities such as the example above. 

While it would be relatively straightforward to hook up a trading logic and portfolio management algorithm to the finder, this algorithm is not intended for actual trading purposes. First, Python is prohibitively slow for trading at sub-second scale. Second, being based on for loops, the algorithm itself is not optimized for speed. Third, one would want to specify on a handful of currencies which are tracked and traded; tracking thousands of assets adds unnecessary computational overhead.  Lastly, the algorithm does not take into account orderbook depth and prices at different depth levels.

-----

**Install and import dependencies**

In [1]:
%pip install ccxt
%pip install websocket-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import ccxt
import requests
import websocket
import json
from pprint import pprint
import time
from datetime import datetime

**Finder class**

The outline of the two main class methods:

**find_paths():** For finding all tradable triangular paths on Binance. This operation is executed only once before the continuous scanning. Hence, speed is not a concern.

* Fetch all tickers on Binance (and filter out derivatives)
* Fetch all traded pairs on Binance
* Initialize order books with placeholders for bid and ask prices for all valid pairs (bid price is highest price buyers are prepared to buy; ask price is the lowest price sellers are willing to sell)
* Brute force through all three-ticker combinations, and save those that constitute a tradeable triangular path.
* Remove duplicate paths (t1 -> t2 -> t3 -> t1 is effectively identical as t2 -> t3 -> t1 -> t2)
* The remaining set of paths constitutes the scanned universe.

**find_arbitrage():** Process data received from websocket and scan for arbitrage opportunities. At every received payload:

* Update orderbooks for each pair with the most recent bid and ask prices
* Loop through all paths, compute the cumulative net return for trading the path, and save profitable paths
* Print all profitable arbitrage paths with trading instructions and resulting net return

See the comments and code for more details.

In [3]:
class TriArbFinder:

    def __init__(self):
        self.paths = list()
        self.books = dict()


    def valid_pair(self, from_tick, to_tick):
        '''
          Helper function for checking whether "from_tick" and "to_tick" constitute a valid
          trading pair, and whether the pair should be bough or sold in order to move 
          from asset "from_tick" to asset "to_tick".
        '''
        is_valid = pair = direction = None

        if from_tick + to_tick in self.books: # Check whether the pair exists in the books containing all valid pairs (= is a tradeable pair)
            pair = from_tick + to_tick        # Record the pair as level 1 trade
            direction = 'sell'                # We sell the pair @ bid to go from 'from_tick' to 'to_tick' (price is numerator)
            is_valid = True                   # Indicate that 'from_tick' and 'to_tick' constitute a valid trading pair

        if to_tick + from_tick in self.books: # Check whether the reverse pair exists in the books (only either one can be)
            pair = to_tick + from_tick
            direction = 'buy'                 # We buy the pair @ ask to go from 'from_tick' to 'to_tick' (price is denominator)
            is_valid = True

        return is_valid, pair, direction


    def find_paths(self):

        t = time.time()

        print('Identifying all triangular paths...')

        # Get all single tickers
        exchange    = ccxt.binance()
        markets     = exchange.load_markets()
        all_tickers = list(exchange.currencies)

        # Filter out derivatives
        derivatives = ('UP', 'DOWN', 'BULL', 'BEAR')  
        tickers     = [ticker for ticker in all_tickers if not (ticker.endswith(derivatives) or ticker == 'T')]

        # Get valid pairs (only a subset of all ticker combinations can be traded on Binance)
        url         = 'https://api.binance.com/api/v3/exchangeInfo'
        resp        = requests.get(url)
        einfo       = resp.json()

        valid_pairs = []
        for pair in einfo['symbols']:
            if pair['status'] == 'TRADING':
                valid_pairs.append(pair['symbol'])

        # Initialize order books for all valid pairs
        for pair in valid_pairs:
            self.books[pair] = {'bid_price': 0, 'ask_price': 0}

        # Find all possible triangular paths (from tick_1 -> tick_2 -> tick_3 -> tick_1)
        # Brute force through all three-pair combinations, and save those that construct valid, tradeable paths
        all_paths = []
        for t1 in tickers:
            for t2 in tickers:
                for t3 in tickers:
                    if not (t1 == t2 or t1 == t3 or t2 == t3): # Ensure the path goes through three distinct assets

                        # Trade 1 / t1 -> t2
                        is_valid1, pair1, direction1 = self.valid_pair(t1, t2)

                        # Trade 2 / t2 -> t3
                        is_valid2, pair2, direction2 = self.valid_pair(t2, t3)

                        # Trade 3 / t3 -> t1
                        is_valid3, pair3, direction3 = self.valid_pair(t3, t1)

                        # Chekc if valid path identified
                        if is_valid1 and is_valid2 and is_valid3:
                            p = {
                                't1': t1,
                                't2': t2,
                                't3': t3,
                                'pair1': pair1,
                                'pair2': pair2,
                                'pair3': pair3,
                                'direction1': direction1,
                                'direction2': direction2,
                                'direction3': direction3,
                                'cum_ret': -100,
                                'path': ''
                                }
                            all_paths.append(p)

        print('All possible triangular paths identified. Proceeding to remove duplicates...')

        # Remove redundant paths
        # E.g., (tick_1 -> tick_2 -> tick_3 -> tick_1) is effectively same as (tick_2 -> tick_3 -> tick_1 -> tick_2)
        # Both loops are identical, with identical return and trades, thus we'll keep only one of these to reduce computational expense
        seen_paths = []
        for p in all_paths:
            
            pot_path = p['t1'] + p['t2'] + p['t3']
            
            if not any(pot_path in s for s in seen_paths):
                
                # Append a new path to the list containing seen paths.
                # t1t2t3 becomes t1t2t3t1t2t3 when we use *2 operator on it.
                # This way the string includes the path's all starting configurations,
                # (t1->t2->t3, t2->t3->t1, and t3->t1->t2), 
                # which makes checking seen paths easy with the single line if statement above
                seen_paths.append(pot_path*2)

                # Append a new path to list containin the final set of paths that will be used by the main algorithm
                self.paths.append(p)

        t = time.time() - t
        print(f'Finished (elapsed {t:.1f} seconds).')
        print(f'Total {len(self.paths)} unique paths identified from {len(tickers)} tickers.')


    def find_arbitrage(self, data, fee=0, net_return_threshold=0):

        # Container for profitable arbitrage opportunities
        arb_paths = []
        
        # Update class orderbooks with the latest snapshot of the market data orderbooks
        for d in data:
            pair = d['s']
            self.books[pair]['bid_price'] = float(d['b'])
            self.books[pair]['ask_price'] = float(d['a'])

        # Loop through all valid triangular trading paths
        for p in self.paths:
            
            if self.books[p['pair1']]['bid_price'] != 0 and \
                self.books[p['pair2']]['bid_price'] !=0 and \
                self.books[p['pair3']]['bid_price'] != 0:
                
                cum_ret = float()
                path_str = str()

                # Trade 1
                if p['direction1']  == 'sell':
                    # SELL the pair @ BID
                    cum_ret  = self.books[p['pair1']]['bid_price']
                    path_str = f"{p['t1']} -> [{p['pair1']}][sell @ bid][{ self.books[p['pair1']]['bid_price'] }] -> {p['t2']}\n"
                else: 
                    # BUY the pair @ ASK
                    cum_ret  = 1 / self.books[p['pair1']]['ask_price']
                    path_str = f"{p['t1']} -> [{p['pair1']}][buy @ ask][{ self.books[p['pair1']]['ask_price'] }] -> {p['t2']}\n"
                
                # Trade 2
                if p['direction2']  == 'sell':
                    cum_ret  *= self.books[p['pair2']]['bid_price']
                    path_str += f"{p['t2']} -> [{p['pair2']}][sell @ bid][{ self.books[p['pair2']]['bid_price'] }] -> {p['t3']}\n"
                else:
                    cum_ret  *= 1 / self.books[p['pair2']]['ask_price']
                    path_str += f"{p['t2']} -> [{p['pair2']}][buy @ ask][{ self.books[p['pair2']]['ask_price'] }] -> {p['t3']}\n"

                # Trade 3
                if p['direction3']  == 'sell':
                    cum_ret  *= self.books[p['pair3']]['bid_price']
                    path_str += f"{p['t3']} -> [{p['pair3']}][sell @ bid][{ self.books[p['pair3']]['bid_price'] }] -> {p['t1']}\n"
                else:
                    cum_ret  *= 1 / self.books[p['pair3']]['ask_price']
                    path_str += f"{p['t3']} -> [{p['pair3']}][buy @ ask][{ self.books[p['pair3']]['ask_price'] }] -> {p['t1']}\n"

                # Append to the path element (i) the path in text format, and (ii) the cumulative roundtrip return for trading the path
                p['path'] = path_str
                p['cum_ret'] = (cum_ret - 3*fee -1)*100

                # Append to profitable arbitrage paths
                if p['cum_ret'] > net_return_threshold*100:
                    arb_paths.append(p)

        # Print profitable paths
        for ap in arb_paths:
            msg = f'''
[{datetime.now()}]
{ap['path']}
Net roundtrip return: {ap['cum_ret']:.3f} %

'''
            print(msg)

**Instantiate TriArbFinder**

In [4]:
finder = TriArbFinder()

**Find all triagular paths**

In [5]:
finder.find_paths()

Identifying all triangular paths...
All possible triangular paths identified. Proceeding to remove duplicates...
Finished (elapsed 180.0 seconds).
Total 5314 unique paths identified from 464 tickers.


**Open socket and run algorithm**

In [8]:
# Define search parameters (specify in decimal format: 0.1% = 0.001)
fee = 0 # Fee per single trade (standard fee on Binance: 0.00075)
net_return_threshold = 0

In [7]:
# Define web socket functions
def on_open(ws):
    print('\nConnection opened...\n')

def on_close(ws):
    print('\nConnection closed.\n')

def on_message(ws, message):
    json_message = json.loads(message)
    finder.find_arbitrage(json_message, fee, net_return_threshold)

In [9]:
# Open socket connection and run algorithm
SOCKET = 'wss://stream.binance.com:9443/ws/!ticker@arr'
ws = websocket.WebSocketApp(SOCKET, 
                            on_open=on_open, 
                            on_close=on_close,
                            on_message=on_message)

ws.run_forever()


Connection opened...


[2022-09-17 14:24:00.665215]
AERGO -> [AERGOBTC][sell @ bid][7.15e-06] -> BTC
BTC -> [BTCBUSD][sell @ bid][19916.66] -> BUSD
BUSD -> [AERGOBUSD][buy @ ask][0.1424] -> AERGO

Net roundtrip return: 0.003 %



[2022-09-17 14:24:00.665380]
ALICE -> [ALICETRY][sell @ bid][37.37] -> TRY
TRY -> [BTCTRY][buy @ ask][368816.0] -> BTC
BTC -> [ALICEBTC][buy @ ask][0.0001013] -> ALICE

Net roundtrip return: 0.024 %



[2022-09-17 14:24:00.665425]
ALICE -> [ALICETRY][sell @ bid][37.37] -> TRY
TRY -> [BUSDTRY][buy @ ask][18.514] -> BUSD
BUSD -> [ALICEBUSD][buy @ ask][2.018] -> ALICE

Net roundtrip return: 0.023 %



[2022-09-17 14:24:00.665461]
BNB -> [BNBBTC][sell @ bid][0.01391] -> BTC
BTC -> [WAVESBTC][buy @ ask][0.0002167] -> WAVES
WAVES -> [WAVESBNB][sell @ bid][0.01558] -> BNB

Net roundtrip return: 0.008 %



[2022-09-17 14:24:00.665511]
BNB -> [BNBUSDC][sell @ bid][277.1] -> USDC
USDC -> [BTCUSDC][buy @ ask][19918.28] -> BTC
BTC -> [BNBBTC][buy @ ask][0.013911] -> BNB


ERROR:websocket: - goodbye
ERROR:websocket:error from callback <function on_close at 0x7fcb26ad5050>: on_close() takes 1 positional argument but 3 were given


True